In [1]:
import cv2
import numpy as np
from scipy.interpolate import interp1d
import argparse
from skimage import io
import matplotlib.pyplot as plt

In [2]:
s_path = './example/source.png'

In [3]:
r_path = './example/ref.png'

In [4]:
def est_cdf(X):
    bins = np.arange(257)
    y = np.histogram(X, bins, density=True)
    cdf_hist = np.cumsum(y[0])
    x_range = y[1][:256]
    P = interp1d(x_range, cdf_hist)
    return P

In [5]:
def transfer(from_im, to_im):
    # Get the cdf's for each array, as well as the inverse cdf for the from_im
    F = est_cdf(to_im)
    G = est_cdf(from_im)
    G_inv = np.interp(F.y, G.y, G.x, left=0.0, right=1.0)

    # Figure out how to map olf values to new values
    mapping = {}
    x_range = np.arange(256)
    for n, i in enumerate(x_range):
        val = F(i)
        xj = G_inv[n]
        xj = round(xj)
        mapping[i] = xj

    # Apply the mapping
    v_map = np.vectorize(lambda x: mapping[x])
    result = v_map(to_im)
    return result 

In [6]:
def transfer_using_colorspace(source, reference, strength = 1.0):
    from_im_cvt = cv2.cvtColor(reference, cv2.COLOR_BGR2LAB)
    to_im_cvt = cv2.cvtColor(source, cv2.COLOR_BGR2LAB)
    result = to_im_cvt.copy()
    # Only apply histogram transfer in a and b channal
    for i in [1,2]:
        result[:,:,i] = transfer(from_im_cvt[:,:,i], to_im_cvt[:,:,i])
        if(i == 1):
            graph = result[:,:,i].copy()
            bins = np.arange(257)
            hist, hist_bin = np.histogram(graph, bins, density=True)
            center = (hist_bin[:-1] + hist_bin[1:]) / 2
            width = 0.7 * (hist_bin[1] - hist_bin[0])
            plt.bar(center, hist, align='center', width=width)
            plt.title('Adjust Histogram of reference alpha channal', fontsize=5)
            plt.savefig("./result/figure_13_change_reference_image_histogram_alpha.png", dpi=300)
            plt.close()
        else:
            graph = result[:,:,i].copy()
            bins = np.arange(257)
            hist, hist_bin = np.histogram(graph, bins, density=True)
            center = (hist_bin[:-1] + hist_bin[1:]) / 2
            width = 0.7 * (hist_bin[1] - hist_bin[0])
            plt.bar(center, hist, align='center', width=width)
            plt.title('Adjust Histogram of reference beta channal', fontsize=5)
            plt.savefig("./result/figure_14_reference_image_histogram_beta.png", dpi=300)
            plt.close()
    return cv2.cvtColor(result, cv2.COLOR_LAB2BGR)*strength + source * (1 - strength)

In [7]:
source = cv2.cvtColor(io.imread(s_path), cv2.COLOR_RGB2BGR)
reference = cv2.cvtColor(io.imread(r_path), cv2.COLOR_RGB2BGR)

result = transfer_using_colorspace(source, reference, strength = 0.8)

In [ ]:
cv2.imwrite('./result/output.jpg', result)